<a href="https://colab.research.google.com/github/zhangxs131/Spider/blob/main/%E5%85%B0%E5%BE%B7%E5%85%AC%E5%8F%B8_spider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#爬取 兰德公司  https://www.rand.org/about.html

网页为静态网页，直接使用requets+lxml进行获取

设置搜索关键词，时间 开始爬取

保持，keyword：data的json文件

data中包括 time，title，url，text 

In [18]:
import requests
import lxml.etree as etree
import datetime
import json
import os
from concurrent.futures import ThreadPoolExecutor
import threading


headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/51.0.2704.63 Safari/537.36'}
pool = ThreadPoolExecutor(max_workers=10)



querys=['Ukraine','Russia','Putin','Volodymyr Zelensky','war']
time='week'#'month'
pp='10'
pages=20
start_time='2022-03-01 00:00:00'
end_time='2022-03-11 00:00:00'

ori_url='https://www.rand.org/search.html?query={}&sortby=date&page={}'

result={}
sum=0

def thread_action(li):
    sample={}

    sample['title']=li.xpath('.//h3[@class="title"]/a/text()')[0].strip()
    sample['url']=li.xpath('.//h3[@class="title"]/a/@href')[0]
    

    #获取文章页面
    try:
      page_res = requests.get(sample['url'], headers = headers, timeout = 30)
      page_res.raise_for_status()
      page_res.encoding = page_res.apparent_encoding

    except:
      print('error in geting text pages')
    t=etree.HTML(page_res.text)

    #blog 页面
    if sample['url'].split('/')[3]=='blog':
      
      sample['text']=t.xpath('//div[@class="body-text"]/p/text()')

      if len(sample['text'])==0:
        return {}
      sample['text']=('\n').join(sample['text'])

      #处理时间

      sample['time']=t.xpath('//meta[@name="citation_online_date"]/@content')[0]
      
    # pubs页面
    elif sample['url'].split('/')[3]=='pubs':
      sample['text']=t.xpath('//div[@class="abstract product-page-abstract"]/p/text()')

      if len(sample['text'])==0:
        return {}
      sample['text']=('\n').join(sample['text'])
      if len(sample['text'])==0:
        return {}

      sample['time']=t.xpath('//meta[@name="citation_online_date"]/@content')[0]


    #晚于3月份的
    if 'time' not in sample:
      return {}

    print(sample['title'])
    return sample


for query in querys:
  print('\n\n____________________________under searching keywork {}_________________________________'.format(query))

  data=[]
  for i in range(pages):
    url=ori_url.format(i+1,query)


    try:
      res = requests.get(url, headers = headers, timeout = 30)
      res.raise_for_status()
      res.encoding = res.apparent_encoding

    except:
      print('error in geting search pages')


    lis = etree.HTML(res.text).xpath('//ul[@class="teasers list organic"]/li')

    for j in lis:
      #多线程
      future= pool.submit(thread_action, j)

      if future.result()=={}:
        continue
      sum+=1
      if sum%10==0:
        print('___________________________Sum: {} News Time: {} ______________________\n'.format(sum,future.result()['time']))

      data.append(future.result())

    if future.result()!={} and int(future.result()['time'].split('/')[1])<3:
      break


  result[query]=data
  data=[]

pool.shutdown()

#save data
with open('rand.json'.format(query),'w') as f:
  f.write(json.dumps(result,ensure_ascii=False))



____________________________under searching keywork Ukraine_________________________________
Observed Impacts, Future Risks, and Adaptation Solutions: Highlights from the Recent Intergovernmental Panel on Climate Change (IPCC) Working Group II Report
Cyber Mission Thread Analysis: A Prototype Framework for Assessing Impact to Missions from Cyber Attacks to Weapon Systems
Cyber Mission Thread Analysis: An Implementation Guide for Process Planning and Execution
'Grow Your Own' Prep Programs Can Improve Diversity of the Teaching Workforce
Standardized Assessment of Depression Symptoms in Post-Acute Care: A Screening Threshold Approach
Lessons Learned Recruiting and Retaining Pregnant and Postpartum Individuals in Digital Trials
Assessing the Effectiveness of Future Concepts in the U.S. Air Force: Application to Future Logistics Concepts
The West's Response to Russia, the State of the Union, Outer Space: RAND Weekly Recap
Competition and Restraint in Cyberspace: The Role of International